### Lab 2 - Build a LSTM model for text generation
#### Basic Steps for LSTM Text Generation
##### 1. Load and Preprocess Data:
- Load the text data from a source.
- Convert the text to lowercase to ensure consistency.

##### 2. Create Character Mappings:
- Identify the unique characters in the text.
- Create dictionaries to map characters to indices and vice versa.

##### 3. Create Input Sequences:
- Define the length of input sequences (maxlen).
- Extract overlapping sequences of text (sentences) and their corresponding next characters (next_chars).

##### 4. Vectorize the Data:
- Convert the sequences and the next characters to one-hot encoded vectors (x for input sequences, y for next characters).

##### 5. Build the LSTM Model:
- Define a sequential model.
- Add LSTM layers and a Dropout layer for regularization.
- Add a Dense layer with a softmax activation function to predict the next character.

##### 6. Compile the Model:
- Compile the model using an optimizer (e.g., Adam) and a loss function (e.g., categorical cross-entropy).

##### 7. Train the Model:
- Train the model on the vectorized data using a specified batch size and number of epochs.
- Optionally, use callbacks to monitor training progress and generate text at intervals.

##### 8. Generate Text:
- Define a function to sample the next character based on model predictions and a temperature parameter.
- Generate text by predicting the next character iteratively, starting from a random seed sequence.

In [3]:
# Import Necessary Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import LambdaCallback
import random
import sys
import time

In [5]:
# This cell of the code is useful if you have a GPU like Nvidia or If you are using a cloud platform like kaggle
# Ensure GPU is being used
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Using GPU")
else:
    print("No GPU found, using CPU")

No GPU found, using CPU


In [7]:
# Load and preprocess the text data
path = tf.keras.utils.get_file('nietzsche.txt', 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path, 'rb').read().decode(encoding='utf-8')
text = text.lower()  # Convert to lowercase

600901/600901 ━━━━━━━━━━━━━━━━━━━━ 2s 3us/step


In [9]:
# Create character-level mappings
chars = sorted(list(set(text)))
char_indices = {char: i for i, char in enumerate(chars)}
indices_char = {i: char for i, char in enumerate(chars)}

In [11]:
# Cut the text into sequences
maxlen = 40  # Length of input sequences
step = 1     # Step size to create sequences
sentences = []  # Input sequences
next_chars = []  # Output characters
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

In [13]:
# Vectorization
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [17]:
# Build the RNN model
model = Sequential([
    LSTM(128, input_shape=(maxlen, len(chars)), return_sequences=True),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(128),
    Dense(len(chars), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [19]:
# Function to sample the next character given the model's predictions
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [21]:
# Function to generate text in each epoch
def on_epoch_end(epoch, _):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence

    print('----- Generating with seed: "' + sentence + '"')
    for i in generated:
        time.sleep(0.05)
        print(i,end="", flush=True)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature=0.5)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
        print(next_char,end="", flush=True)
    print()

# Callback to generate text after each epoch
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
# Train the model
model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
4695/4695 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - loss: 2.6227
----- Generating text after Epoch: 0
----- Generating with seed: "many sentiments are lost to us is manife"
many sentiments are lost to us is manife condere the gorenthing and the more the pance the ham the pondiculle with the nend to ke ness to the end we the has his be and and to this the conderest of a condist the soming fect to with be the was distert of the everenty in the mento in the soment penconce of the the ress and to the ferem and is to the for to he suftes. which the for the pongence and their and ow be pocenting the and seal of 
4695/4695 ━━━━━━━━━━━━━━━━━━━━ 973s 206ms/step - loss: 2.6226
Epoch 2/10
4695/4695 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step - loss: 1.9772
----- Generating text after Epoch: 1
----- Generating with seed: "presses his way to the tragedy, the work"
presses his way to the tragedy, the work, the spectaring of the were
which the more the porsent of the dore in the mort merse of a so for d

In [ ]:
model.save("LSTM for Text generation 90epocs 3 LSTM layers.h5")

In [ ]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import LambdaCallback
import random
import sys
import time

# Setup PlaidML
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import plaidml.keras
plaidml.keras.install_backend()

# Ensure GPU is being used
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Using GPU")
else:
    print("No GPU found, using CPU")

# Load and preprocess the text data
path = tf.keras.utils.get_file('nietzsche.txt', 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path, 'rb').read().decode(encoding='utf-8')
text = text.lower()  # Convert to lowercase

# Create character-level mappings
chars = sorted(list(set(text)))
char_indices = {char: i for i, char in enumerate(chars)}
indices_char = {i: char for i, char in enumerate(chars)}

# Cut the text into sequences
maxlen = 40  # Length of input sequences
step = 1     # Step size to create sequences
sentences = []  # Input sequences
next_chars = []  # Output characters
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

# Vectorization
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# Build the RNN model
model = Sequential([
    LSTM(128, input_shape=(maxlen, len(chars)), return_sequences=True),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(128),
    Dense(len(chars), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Function to sample the next character given the model's predictions
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Function to generate text in each epoch
def on_epoch_end(epoch, _):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence

    print('----- Generating with seed: "' + sentence + '"')
    for i in generated:
        time.sleep(0.05)
        print(i, end="", flush=True)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature=0.5)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
        print(next_char, end="", flush=True)
    print()

# Callback to generate text after each epoch
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# Train the model
model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])


ImportError: cannot import name 'conv_utils' from 'keras.utils' (c:\Users\bhawa\miniconda3\envs\tester\Lib\site-packages\keras\api\utils\__init__.py)